<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%206%20-%20Lesson%203%20-%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
# 전이 학습. 쉽게 말해 남이 만들어놓은 모델 가져와서 내 입맛에 맞게 설정하는 방식이다. 
# 엄청나게 학습된 애들을 그대로 가져와 쓰니까 성능 확실하다. 이걸 다시 학습시킬 수도 있고 그대로 쓸 수도 있다. 학습 정도도 다 설정할 수 있다. 
# 보통 합성곱 부분은 그대로 두고 막판에 연결하는 완전 연결 레이어들만 따로 훈련시키는 것 같다. 왜냐하면 합성곱 부분까지 훈련시키려면 
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3 # 훈련된 모델 가져오기

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, # 원래 V3 모델의 입력 쪽에 Dense 레이어가 있는데 False를 누르면 얘가 사라진다. 
                                weights = None) # 가중치 설정. none이면 설정 X

pre_trained_model.load_weights(local_weights_file) # 이렇게 가중치를 불러옴

for layer in pre_trained_model.layers:
  layer.trainable = False # 가저온 레이어들을 다시 훈련시킬 것인가? False는 아니오. 왜냐하면 얘들을 훈련시켰던 데이터들은 크기가 웅장한 애들인데 내가 갖고 있는 데이터는 쥐꼬리 수준이니 재훈련 시키면 역효과가 나고 그러면 모델을 가져온 이유가 없다. 
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7') # 가져온 모델의 마지막 층...은 아니고 마지막 직전 층까지 모델을 첫 레이어부터 쫘아악 가져옴. 최종 출력은 3*3인데 얘는 그 이전 층인 7*7을 출력해준다. 
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output 

--2021-01-19 18:13:11--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 34.64.4.80, 34.64.4.16
Connecting to storage.googleapis.com (storage.googleapis.com)|34.64.4.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: `/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

/tmp/inception_v3_w 100%[===================>]  83.84M  7.92MB/s    in 18s     

2021-01-19 18:13:29 (4.73 MB/s) - `/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5' saved [87910968/87910968]

last layer output shape:  (None, 7, 7, 768)


In [0]:
from tensorflow.keras.optimizers import RMSprop

# 훈련된 모델에 이어줄 모델을 생성(분류 모델)
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2. 앞서 추가한 1024개의 뉴런 중 20%를 매 훈련마다 임의로 선택해 비활성화 하며 골고루 훈련시켜준다. 그러면 과대적합 현상을 막을 수 있다. 
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x) # 이진 분류니까 시그모이드 활성화 함수를 사용한다.          

model = Model( pre_trained_model.input, x) # Model abstract class를 사용해 모델 생성. 기존 모델들로 새 모델을 만드는 방식인듯 하다. 

model.compile(optimizer = RMSprop(lr=0.0001), # 최적화 방식, 손실 함수 등 설정)
              loss = 'binary_crossentropy', 
              metrics = ['accuracy']) 


In [0]:
# 데이터 가져오기

!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# 데이터 전처리
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

In [0]:
# 훈련
history = model.fit(
            train_generator,
            validation_data = validation_generator, # 어떤 매개변수에 값을 전달할 것인가 명시만 해주면 순서는 상관없다
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

In [0]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()